In [129]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

def write_answer(n, value):
    with open('{}.txt'.format(n), 'w') as f:
        f.write(str(value))

In [79]:
def get_checkins_df():
    data = []
    columns = None
    with open('checkins.dat', 'r') as f:
         for i, line in enumerate(f.readlines()):
            words = [word.strip() for word in line.split('|')]

            if i == 0:
                columns = words
                continue
            elif i == 1:
                continue 
            elif i > 1 and (len(words) == 6 and words[3] != '' and words[4] != ''):
                data.append(words)

    return pd.DataFrame(columns=columns, data=np.array(data))

In [54]:
df = get_checkins_df()
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
2,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [94]:
coordinates = df.iloc[:100000, 3:5].apply(pd.to_numeric)
ms = MeanShift(bandwidth=0.1)
ms.fit(coordinates.as_matrix(), n_jobs=-1)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [102]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique, labels_counts = np.unique(labels, return_counts=True)
filtered_labels, filtered_centers = [], []
for label, count in zip(labels_unique, labels_counts):
    if count > 15:
        filtered_labels.append(label)
        filtered_centers.append(cluster_centers[label])
filtered_centers

[array([ 40.7177164 , -73.99183542]),
 array([  33.44943805, -112.00213969]),
 array([  33.44638027, -111.90188756]),
 array([ 41.87824378, -87.62984336]),
 array([  37.68868157, -122.40933037]),
 array([ 38.88616522, -77.04878333]),
 array([  33.35734456, -111.82265411]),
 array([ 33.76663623, -84.39328918]),
 array([ 42.36321864, -71.07368761]),
 array([  47.60624472, -122.33204383]),
 array([  36.11722914, -115.17107342]),
 array([  34.06039755, -118.24870903]),
 array([ 44.97794782, -93.26730089]),
 array([ 30.26718362, -97.74311928]),
 array([ 40.76687624, -73.83335349]),
 array([  39.73583015, -104.98658043]),
 array([ 39.95168037, -75.16273592]),
 array([  34.03548695, -118.43899772]),
 array([  32.98089338, -117.07811798]),
 array([ 32.80302054, -96.76989743]),
 array([  37.34787114, -121.94728722]),
 array([ 28.54350155, -81.37642862]),
 array([  32.71134443, -117.15363875]),
 array([  32.22171315, -110.92653515]),
 array([  34.12740222, -118.35188373]),
 array([ 29.76269775, 

In [119]:
addresses_df = pd.read_csv('addresses.csv')
addresses_df

,latitude,longtitude,city
0,33.751277,-118.188740,Los Angeles
1,25.867736,-80.324116,Miami
2,51.503016,-0.075479,London
3,52.378894,4.885084,Amsterdam
4,39.366487,117.036146,Beijing
5,-33.868457,151.205134,Sydney


In [101]:
def get_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

In [131]:
nearest_centers = []
min_distances = []
for center in filtered_centers:
    min_distance, nearest_center = None, None
    for address in addresses_df[['latitude', 'longtitude']].as_matrix():
        distance = get_distance(np.array(center), address)
        if min_distance is None or distance < min_distance:
            min_distance, nearest_center = distance, center
    nearest_centers.append(nearest_center)
    min_distances.append(min_distance)

sorted_distances_indexes = np.argsort(min_distances)
sorted_centers = np.array(nearest_centers)[sorted_distances_indexes]
print 'The nearest cluster center is {}'.format(sorted_centers[0])
print 'Min distance is {}'.format(min_distances[sorted_distances_indexes[0]])
write_answer(1, '{} {}'.format(sorted_centers[0][0], sorted_centers[0][1]))

The nearest cluster center is [ -33.86063043  151.20477593]
Min distance is 0.00783475816311
